In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('dataset1.csv', encoding='cp1251', sep='\t', header=None)
df2 = pd.read_csv('dataset2.csv', encoding='cp1251', sep='\t', header=None, quoting=3)
df = pd.concat([df1, df2], axis=0).reset_index()

In [3]:
df.head()

,index,0,1,2,3,4,5,6,7,8
0,0,B057586722045,_ЁЭффективное численное решение задачи Стокса ...,_ЁРассматривается задача Стокса с разрывным ко...,_ЁХабаровск\_Ёзадача Стокса с разрывным коэффи...,e8\f2,13Д\16Б,27.41.19\30.17.02,271.41.19.17.21\301.17.02.11,###
1,1,B10213980294,_ЁАвтоматическая маршрутизация в среде с препя...,_ЁВ работе рассматривается перемещение тела гр...,_Ёавтоматическая маршрутизация\_Ёинтеллектуаль...,e8\e1\e9\00,93\93-Y\37\37-Y\РС\81,28.23.27\50.41.21\55.30.31,282.23.27.09\509.41.21.25.19\551.30.31.07.13,###
2,2,B103249601493,Структурно-динамический анализ с помощью ЯМР-р...,Изучение движения молекул и атомов в твердых т...,Татарстан\ЯМР-релаксометрия\газы природные\доб...,e5\f7,08Д\19А,31.01.33\38.57.23,311.01.33\383.57.23.01,###
3,3,B10349920193X,Структурно-функциональные исследования пептидо...,Излагается разрабатываемая концепция создания ...,изучение\лекарственные средства\пептиды\разраб...,e3,04Р1\04Б3,62.13.39,621.13.39,###
4,4,B103499202650,Применение хроматографии и масс-спектрометрии ...,В результате проведенных исследований разработ...,Galleria mellonella\антибактериальная активнос...,e3,04Р1\04Б3,62.13.39,621.13.39,###


In [4]:
txt = open('viniti_md.txt','r',encoding='cp1251',)
tx = txt.readlines()
alphabet = []
for x in tx:
    alphabet.append(x.strip('\n').split('\t'))
    
alpha_df = pd.DataFrame(alphabet, columns=['символ',"значение", "код", "категория"])

def valid_alpha(grp:list) -> list:
    unvalid_alpha = alpha_df.copy()
    unvalid_alpha.drop(unvalid_alpha.index[unvalid_alpha['категория'].isin(grp)], inplace=True)
    return sorted(unvalid_alpha['символ'], key=lambda x: len(x), reverse=True)

unvalid_alpha = valid_alpha(['буквы лат.','цифры','буквы рус.', 'пробел'])
for allowed_el in ['-', '.', '!', ',', ':', ';', '?']:
    unvalid_alpha.remove(allowed_el)

In [5]:
def text_processing(unvalid_alpha: list, s: str)->str:
    current_str = s
    for x in unvalid_alpha:
        current_str = current_str.replace(x, ' ')
    tokens = current_str.split(' ')
    current_str = ' '.join([token for token in tokens if token != ''])
        
    return current_str

In [6]:
%%time
name = 1
anot = 2
keywords = 3

df['keywords'] = df[keywords].apply(lambda x: list(map(lambda y: text_processing(unvalid_alpha, y), x.split('\\'))))
df['name'] = df[name].apply(lambda x: text_processing(unvalid_alpha, x))
df['anot'] = df[anot].apply(lambda x: text_processing(unvalid_alpha, x))

Wall time: 50.8 s


In [7]:
df_cleared = df[['keywords', 'name', 'anot']]
df_cleared.head()

,keywords,name,anot
0,"[Хабаровск, задача Стокса с разрывным коэффици...",Эффективное численное решение задачи Стокса с ...,Рассматривается задача Стокса с разрывным коэф...
1,"[автоматическая маршрутизация, интеллектуальны...",Автоматическая маршрутизация в среде с препятс...,В работе рассматривается перемещение тела груп...
2,"[Татарстан, ЯМР-релаксометрия, газы природные,...",Структурно-динамический анализ с помощью ЯМР-р...,Изучение движения молекул и атомов в твердых т...
3,"[изучение, лекарственные средства, пептиды, ра...",Структурно-функциональные исследования пептидо...,Излагается разрабатываемая концепция создания ...
4,"[Galleria mellonella, антибактериальная активн...",Применение хроматографии и масс-спектрометрии ...,В результате проведенных исследований разработ...


In [ ]:
df_cleared['name_anot'] = df_cleared.apply(lambda x: x[1]+'. '+ x[2], axis=1)

In [14]:
df_exp = df_cleared.sample(10000).reset_index()
df_exp.to_csv('df_exp.csv', index=False)

### Rake

In [8]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from rake_nltk import Rake

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ilya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ilya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
def get_stopwords_list(add_external=True):
    nltk_list = nltk.corpus.stopwords.words('russian') 
    with open('stopwords-ru.txt', 'r', encoding='utf-8') as f:
        external_list = f.read().split('\n')
    final_list = list(set(nltk_list) | set(external_list)) if add_external == True else nltk_list
        
    return final_list

russian_stopwords = get_stopwords_list()

In [168]:
def predict_keyphrases(text, model, n_words=5):
    if model == 'RAKE':
        rake.extract_keywords_from_text(text)
        keyphrases = rake.get_ranked_phrases()[:n_words]
        
    if model == 'YAKE':
        keyphrases = list(map(lambda x: x[0], yake.extract_keywords(text)))
    
    return keyphrases[:n_words]

In [250]:
%%time
rake = Rake(stopwords=russian_stopwords, max_length=3)
df_exp['keyw_RAKE'] = df_exp['name_anot'].apply(lambda x: predict_keyphrases(x, 'RAKE', 10))

Wall time: 4.39 s


In [251]:
from nltk.stem.snowball import SnowballStemmer

def stem_phrase(phrase):
    stemmer = SnowballStemmer("russian")  
    words = phrase.lower().split(' ')
    stemmed_phrase = ' '.join([stemmer.stem(word) for word in words])
    
    return stemmed_phrase

def check_occurence(check_phrase, phrases_list):
    for phrase in phrases_list:
        k = 0
        phrase_granulated = set(phrase.split(' '))
        check_phrase_granulated = check_phrase.split(' ')
        for check_word in check_phrase_granulated:
            if check_word in phrase_granulated:
                k += 1
        if k/len(check_phrase_granulated) > 0.5:
            return True
    
    return False
    
def keywords_precision_score(keyw_true, keyw_pred, k=5):
    if len(keyw_true) == 0 or len(keyw_pred) == 0:
        return 0
        
    n = 0
    keyw_true = [stem_phrase(phrase) for phrase in keyw_true]
    keyw_pred = [stem_phrase(phrase) for phrase in keyw_pred[:k]]
        
    for keyw in keyw_pred:
        if check_occurence(keyw, keyw_true): 
            n+=1
    
    return n/k

def keywords_recall_score(keyw_true, keyw_pred, k=5):
    if len(keyw_true) == 0 or len(keyw_pred) == 0:
        return 0
    
    n_match = 0
    n_true = len(keyw_true)
    keyw_true = [stem_phrase(phrase) for phrase in keyw_true]
    keyw_pred = [stem_phrase(phrase) for phrase in keyw_pred[:k]]
        
    for keyw in keyw_true:
        if check_occurence(keyw, keyw_pred): 
            n_match += 1
    
    return n_match/n_true

def keywords_mean_reciprocal_rank(keyw_true, keyw_pred, k=5):
    if len(keyw_true) == 0 or len(keyw_pred) == 0:
        return 0
    
    n = 0
    keyw_true = [stem_phrase(phrase) for phrase in keyw_true]
    keyw_pred = [stem_phrase(phrase) for phrase in keyw_pred[:k]]
        
    for n_el, keyw in enumerate(keyw_pred):
        if check_occurence(keyw, keyw_true): 
            n = n_el + 1
            break
    
    return 1/n if n != 0 else 0

def keywords_mean_average_precision(keyw_true, keyw_pred, k):
    if len(keyw_true) == 0 or len(keyw_pred) == 0:
        return 0
    
    idxs = []
    keyw_true = [stem_phrase(phrase) for phrase in keyw_true]
    keyw_pred = [stem_phrase(phrase) for phrase in keyw_pred[:k]]
        
    for n_el, keyw in enumerate(keyw_pred):
        if check_occurence(keyw, keyw_true):
            idxs.append(n_el + 1)
            
    if len(idxs) == 0:
        return 0
    
    precisions = [(n_el+1)/ idx for n_el, idx in enumerate(idxs)]
    return sum(precisions)/len(precisions)

def keywords_stem(keyw_true, keyw_pred, k=5):
    n = 0
    keyw_true = [stem_phrase(phrase) for phrase in keyw_true]
    keyw_pred = [stem_phrase(phrase) for phrase in keyw_pred[:k]]
    
    return keyw_true, keyw_pred

def evaluate_extractor(y_true, y_pred, metric, k=5):
    metric_values = [] 
    
    try:
        if metric == 'precision':
            for idx in range(len(y_true)):
                value = keywords_precision_score(y_true[idx], y_pred[idx], k)
                metric_values.append(value)

        if metric == 'recall':
            for idx in range(len(y_true)):
                value = keywords_recall_score(y_true[idx], y_pred[idx], k)
                metric_values.append(value)

        if metric == 'MRR':
            for idx in range(len(y_true)):
                value = keywords_mean_reciprocal_rank(y_true[idx], y_pred[idx], k)
                metric_values.append(value)

        if metric == 'MAP':
            for idx in range(len(y_true)):
                value = keywords_mean_average_precision(y_true[idx], y_pred[idx], k)
                metric_values.append(value)
            
        return np.mean(metric_values)
    
    except KeyError:
        print(y_true[idx])

In [252]:
def extraction_evaluation_report(df, keyw_true_col, models, metrics, size=0.1):
    result = dict()
    df = df.sample(round(size*len(df))).reset_index()
    
    for model_name in models:
        pred_column = 'keyw_' + model_name
        metrics_values = dict()
        for metric_name in metrics:
            metrics_values[metric_name+'_at_5'] = evaluate_extractor(df[keyw_true_col], df[pred_column], metric_name, k=5)
            metrics_values[metric_name+'_at_10'] = evaluate_extractor(df[keyw_true_col], df[pred_column], metric_name, k=10)
        
        result[model_name] = metrics_values
        
    return result

### !pip install yake

In [253]:
import yake

yake = yake.KeywordExtractor (
    lan = "ru",     # язык
    n = 3,          # максимальное количество слов в фразе
    dedupLim = 0.3, # порог похожести слов
    top = 10        # количество ключевых слов
)

In [254]:
%%time
df_exp['keyw_YAKE'] = df_exp['name_anot'].apply(lambda x: predict_keyphrases(x, 'YAKE', 10))

Wall time: 14min 12s


In [255]:
eval_result = extraction_evaluation_report(df_exp,
                                           'keywords',
                                           ['RAKE', 'YAKE'],
                                           ['precision', 'recall', 'MRR', 'MAP'],
                                           size=1)

In [256]:
for model in eval_result.keys():
    print("Extractor:", model)
    for metric in eval_result[model].keys():
        print('\t', metric, ":", round(eval_result[model][metric], 3))

Extractor: RAKE
	 precision_at_5 : 0.103
	 precision_at_10 : 0.085
	 recall_at_5 : 0.174
	 recall_at_10 : 0.24
	 MRR_at_5 : 0.196
	 MRR_at_10 : 0.213
	 MAP_at_5 : 0.194
	 MAP_at_10 : 0.206
Extractor: YAKE
	 precision_at_5 : 0.242
	 precision_at_10 : 0.201
	 recall_at_5 : 0.329
	 recall_at_10 : 0.397
	 MRR_at_5 : 0.426
	 MRR_at_10 : 0.445
	 MAP_at_5 : 0.406
	 MAP_at_10 : 0.387


In [257]:
import json
with open('eval_result.json', 'w') as fp:
    json.dump(eval_result, fp)

### !pip install summa

In [273]:
from summa import keywords

def predict_keyphrases(text, model, n_words=5):
    if model == 'RAKE':
        rake.extract_keywords_from_text(text)
        keyphrases = rake.get_ranked_phrases()
        
    if model == 'YAKE':
        keyphrases = list(map(lambda x: x[0], yake.extract_keywords(text)))
        
    if model == 'TextRank':
        cleaned_text = ' '.join([word.replace('\n', '') for word in text.split(' ') if word.lower() not in russian_stopwords])
        keyphrases = keywords.keywords(text, language = "russian").split("\n")
    
    return keyphrases[:n_words]

In [274]:
%%time
df_exp['keyw_TextRank'] = df_exp['name_anot'].apply(lambda x: predict_keyphrases(x, 'TextRank', 10))

Wall time: 2min 24s


In [275]:
%%time
eval_result = extraction_evaluation_report(df_exp,
                                           'keywords',
                                           ['TextRank'],
                                           ['precision', 'recall', 'MRR', 'MAP'],
                                           size=1)

Wall time: 1min 13s


In [276]:
for model in eval_result.keys():
    print("Extractor:", model)
    for metric in eval_result[model].keys():
        print('\t', metric, ":", round(eval_result[model][metric], 3))

Extractor: TextRank
	 precision_at_5 : 0.272
	 precision_at_10 : 0.212
	 recall_at_5 : 0.099
	 recall_at_10 : 0.143
	 MRR_at_5 : 0.404
	 MRR_at_10 : 0.422
	 MAP_at_5 : 0.409
	 MAP_at_10 : 0.417


### KeyBert - DeepPavlov

In [12]:
#!pip install keyphrase-vectorizers --user
#!pip install keybert

In [13]:
from keyphrase_vectorizers import KeyphraseCountVectorizer
from keybert import KeyBERT

In [83]:
df_exp_bert = df_cleared.sample(1000).reset_index()
df_exp_bert.to_csv('df_exp_bert.csv', index=False)

In [69]:
bert_kw = KeyBERT(model='DeepPavlov/rubert-base-cased-sentence')
#bert_kw = KeyBERT(model='miemBertProject/miem-scibert-linguistic') 
#bert_kw = KeyBERT(model='paraphrase-multilingual-MiniLM-L12-v2')

No sentence-transformers model found with name C:\Users\Ilya/.cache\torch\sentence_transformers\DeepPavlov_rubert-base-cased-sentence. Creating a new one with MEAN pooling.


In [84]:
def predict_keyphrases(text, model, n_words=5):
    if model == 'RAKE':
        rake.extract_keywords_from_text(text)
        keyphrases = rake.get_ranked_phrases()
        
    if model == 'YAKE':
        keyphrases = list(map(lambda x: x[0], yake.extract_keywords(text)))
        
    if model == 'TextRank':
        cleaned_text = ' '.join([word.replace('\n', '') for word in text.split(' ') if word.lower() not in russian_stopwords])
        keyphrases = keywords.keywords(text, language = "russian").split("\n")
        
    if model == 'BERT_spacy':
        vectorizer = KeyphraseCountVectorizer(spacy_pipeline="ru_core_news_sm",
                                              stop_words=russian_stopwords,
                                              pos_pattern='<ADJ.*>*<N.*>+')
        bert_keywords = kw_model.extract_keywords(docs=text,
                                                  vectorizer=vectorizer,
                                                  nr_candidates=20,
                                                  top_n=10,
                                                  use_mmr=True,
                                                  diversity=0.3,
                                                 )
        keyphrases = list(map(lambda x: x[0], bert_keywords))
        
    if model == 'BERT_sklearn':
        bert_keywords = kw_model.extract_keywords(docs=text,
                                                  keyphrase_ngram_range=(1,3),
                                                  stop_words=russian_stopwords,
                                                  nr_candidates=20,
                                                  top_n=10,
                                                  use_mmr=True,
                                                  diversity=0.3)
        keyphrases = list(map(lambda x: x[0], bert_keywords))
    
    return keyphrases[:n_words]

In [85]:
%%time
df_exp_bert['keyw_BERT_spacy'] = df_exp_bert['name_anot'].apply(lambda x: predict_keyphrases(x, 'BERT_spacy', 10))

Wall time: 35min 34s


In [ ]:
%%time
df_exp_bert['keyw_BERT_sklearn'] = df_exp_bert['name_anot'].apply(lambda x: predict_keyphrases(x, 'BERT_sklearn', 10))

In [ ]:
%%time
eval_result = extraction_evaluation_report(df_exp_bert,
                                           'keywords',
                                           ['BERT_spacy', 'BERT_sklearn'],
                                           ['precision', 'recall', 'MRR', 'MAP'],
                                           size=1)

In [ ]:
for model in eval_result.keys():
    print("Extractor:", model)
    for metric in eval_result[model].keys():
        print('\t', metric, ":", round(eval_result[model][metric], 3))

### KeyBert - MiemLinguistic

In [ ]:
bert_kw = KeyBERT(model='miemBertProject/miem-scibert-linguistic')

In [ ]:
%%time
df_exp_bert['keyw_BERT_spacy'] = df_exp_bert['name_anot'].apply(lambda x: predict_keyphrases(x, 'BERT_spacy', 10))

In [ ]:
%%time
df_exp_bert['keyw_BERT_sklearn'] = df_exp_bert['name_anot'].apply(lambda x: predict_keyphrases(x, 'BERT_sklearn', 10))

In [ ]:
%%time
eval_result = extraction_evaluation_report(df_exp_bert,
                                           'keywords',
                                           ['BERT_spacy', 'BERT_sklearn'],
                                           ['precision', 'recall', 'MRR', 'MAP'],
                                           size=1)

In [ ]:
for model in eval_result.keys():
    print("Extractor:", model)
    for metric in eval_result[model].keys():
        print('\t', metric, ":", round(eval_result[model][metric], 3))

### KeyBert paraphrase-multilingual-MiniLM-L12-v2

In [ ]:
bert_kw = KeyBERT(model='paraphrase-multilingual-MiniLM-L12-v2')

In [ ]:
%%time
df_exp_bert['keyw_BERT_spacy'] = df_exp_bert['name_anot'].apply(lambda x: predict_keyphrases(x, 'BERT_spacy', 10))

In [ ]:
%%time
df_exp_bert['keyw_BERT_sklearn'] = df_exp_bert['name_anot'].apply(lambda x: predict_keyphrases(x, 'BERT_sklearn', 10))

In [ ]:
%%time
eval_result = extraction_evaluation_report(df_exp_bert,
                                           'keywords',
                                           ['BERT_spacy', 'BERT_sklearn'],
                                           ['precision', 'recall', 'MRR', 'MAP'],
                                           size=1)

In [ ]:
for model in eval_result.keys():
    print("Extractor:", model)
    for metric in eval_result[model].keys():
        print('\t', metric, ":", round(eval_result[model][metric], 3))